In [1]:
import json
import pandas as pd

In [2]:
jet_stream_data = json.load(open("jet_stream_data.json"))

In [3]:
data_dict = dict()
for wind_component_data in jet_stream_data:
    key_name = wind_component_data["header"]["parameterNumberName"]
    key_value = pd.Series(wind_component_data["data"])
    data_dict[key_name] = key_value

In [4]:
jet_stream_dataframe = pd.DataFrame(data_dict)

In [5]:
longitudes = list()
for i in range(181):
    longitudes.extend(list(range(360)))

In [6]:
latitudes = list()
for i in range(90, -91, -1):
    latitudes.extend(list([i]*360))

In [7]:
assert len(latitudes) == len(longitudes)
assert len(latitudes) == len(jet_stream_dataframe)

In [8]:
jet_stream_dataframe["lng"] = longitudes
jet_stream_dataframe["lat"] = latitudes

In [9]:
import json

geojson = {"type": "FeatureCollection", "features": []}

for _, row in jet_stream_dataframe.iterrows():
    feature = {
        "type": "Feature", 
        "geometry": {
            "type": "Point", 
            "coordinates": [row['lng'], row['lat']]
        }, 
        "properties": {
            # https://stackoverflow.com/questions/46962288/change-longitude-from-180-to-180-to-0-to-360
            "longitude": ((row['lng'] - 180) % 360) - 180,
            "latitude": row['lat'],
            "U-component_of_wind": row['U-component_of_wind'],
            "V-component_of_wind": row['V-component_of_wind'],
        }
    }
    geojson['features'].append(feature)

with open('abms/data/jet_stream_data.geojson', 'w') as fp:
    json.dump(geojson, fp, indent=True)    

In [10]:
jet_stream_dataframe["lat"].min()

-90